In [4]:
import pandas as pd
import pyarabic.araby as araby
import nltk
from nltk.corpus import stopwords
import re
from nltk.corpus import stopwords
from nltk.stem.isri import ISRIStemmer
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\win10\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Functions

In [5]:
# data preprocessing Functions
stopWords = stopwords.words('arabic')


def normalize_text(text):
    text = araby.normalize_alef(text) # change إأآا to ا
    text = araby.normalize_teh(text) # change ة to ه
    text = araby.normalize_hamza(text) # change ئ أ ؤ to ء
    text = text.replace("ى", 'ي') # replace ى with ي  BTW the parameteres written in reverse 
    text = text = text.replace("گ", "ك") # replace "گ" with ك parameteres written in reverse 

    return text

special_char = ["!",'"',"#","%","&","'","(",")",
                "+",",","-",".","/",":","<","[",
                "\\","]","*","{","|","}","-","_","0",
                "1", '2','3','4','5','6','7','8','9']

arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''

def clean_text(text):
    
    for char in special_char:
        text = text.replace(char,' ') # Cleaning special Characters
    
    text = text.translate(str.maketrans('','', arabic_punctuations)) # remove punctuation
    text = text.replace('\n', '') # removes line breaks
    text = re.sub(r"^\s+", "", text) # tirms left space
    text = re.sub(r"\s+$", "", text) # trims right space
    text = re.sub(' +', ' ',text) # tirms any space more than one
    text = ' '.join([word for word in text.split() if word not in stopwords.words("arabic")]) # remove stopwords 
    text = text.strip()
    text = araby.strip_diacritics(text) # removing حركات
    
    return text

def light_stem(text):
    words = text.split()
    result = list()
    stemmer = ISRIStemmer()
    for word in words:
        word = stemmer.norm(word, num=1)      # remove diacritics which representing Arabic short vowels
        if not word in stemmer.stop_words:    # exclude stop words from being processed
            word = stemmer.pre32(word)        # remove length three and length two prefixes in this order
            word = stemmer.suf32(word)        # remove length three and length two suffixes in this order
            word = stemmer.waw(word)          # remove connective ‘و’ if it precedes a word beginning with ‘و’
            word = stemmer.norm(word, num=2)  # normalize initial hamza to bare alif
        result.append(word)
    return ' '.join(result)


In [6]:
df = pd.read_csv('new_poems_5top_v2.csv')  
df['poem_text'] = df['poem_text'].apply(lambda x: clean_text(x)) # Cleaning poem text
df['poem_text'] = df['poem_text'].apply(lambda x: light_stem(x)) # Normalizing poem text
df['poem_text'] = df['poem_text'].apply(lambda x: normalize_text(x)) # Stemming poem text

In [ ]:
df.to_csv('new_poems_5top_CNS2.csv', index = False) # Cleaned, Normalized and Stemmed poems.  2 number addressing version 2